# 使用 tensorflow

从前例可以看出训练神经网络实际上是在两个层次上进行.

低级别层次上训练神经网络实际上是对张量的各种操作,如果用 tensorflow 的 api 说明

- 张量,包括存储网络状态的特殊张量.
- 张量操作,像是张量的加减 relu matmul
- 反向传播(backpropagation),一种链式计算梯度的方法.(tensorflow 中使用 GradientTape 类处理)

高层次上以 keras 的 api 说明

- layers,许多的层组合成一个神经网络模型 model
- 损失函数,定义了训练中的反馈信号.
- 优化器,定义了训练中的参数更新方法,如何进行学习.
- 指标,用来评估模型,例如模型的准确度等.
- 循环训练,执行小批量的随机梯度下降.


## 张量变量


In [1]:
import tensorflow as tf
x = tf.ones(shape=(2,1))
print(x)
x = tf.zeros(shape=(2,1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


初始化 shape(2,1) 的张量,全1 或 全0


In [4]:
x = tf.random.normal(shape=(3, 1), mean=0., stddev=1.)
print(x)

x = tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)
print(x)

tf.Tensor(
[[-1.3895277 ]
 [-0.12083016]
 [ 0.7774914 ]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[0.96545494]
 [0.168015  ]
 [0.59226465]], shape=(3, 1), dtype=float32)


随机填充值

- `tf.random.normal(shape=(3, 1), mean=0., stddev=1.)` 是从平均数0和标准差1中产生随机数.等同于 `np.random.normal(size=(3, 1), loc=0.,.scale=1.)`
- `tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)` 是从0到1的均匀分布中产生随机数.等同于 `np.random.uniform(size=(3, 1), low=0., high=1.)`


In [5]:
import numpy as np
x = np.ones(shape=(2, 2))
x[0, 0] = 0.

In [6]:
x = tf.ones(shape=(2, 2))
x[0, 0] = 0.

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

**tensorflow 创建的数组是不可变的**,而 numpy 创建的数组是可变的.

tensorflow 中创建可以修改的张量是靠 tf.Variable 类,使用 tf.Variable 需要提供 initial_value (张量初始值).


In [11]:
v = tf.Variable(initial_value=tf.random.normal(shape=[3, 1], mean=0, stddev=1))
v

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-1.4414899],
       [-0.6730556],
       [ 1.8196104]], dtype=float32)>

创建一个 tf.Variable


In [17]:
v.assign(tf.ones((3, 1)))
print(v)
v[0, 0].assign(3.)
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>


修改 tf.Variable 需要通过 assign 方法.

修改子集也是通过 assign 方法.


In [20]:
v.assign_add(tf.ones((3,1)))
print(v)

v.assign_sub(tf.ones((3,1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [2.],
       [2.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>


assign_add 相当于 +=

assign_sub 相当于 -=


## 张量运算


In [22]:
a = tf.ones((2, 2))
b = tf.square(a)
print(b)
c = tf.sqrt(a)
print(c)
d = b + c
print(d)
e = tf.matmul(a, b)
print(e)
e *= d
print(e)

tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.99999994 0.99999994]
 [0.99999994 0.99999994]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[4. 4.]
 [4. 4.]], shape=(2, 2), dtype=float32)


张量运算

- tf.square 平方
- tf.sqrt 平方根
- `+` 张量相加
- `tf.matmul` 张量乘法
- `*` 张量逐元素相乘


## 再探 GradientTape

从上面的张量运算来看,tensorflow 做了和 numpy 非常相似的工作.但是在 tensorflow 有一条是 numpy 做不到的,tensorflow 可以方便计算任何可微表达式的梯度.只要借助 GradientTape api.


In [27]:
input_var = tf.Variable(initial_value=3.)

with tf.GradientTape() as tape:
    result = tf.square(input_var)
gradient = tape.gradient(result, input_var)

print(gradient)

tf.Tensor(6.0, shape=(), dtype=float32)


最常见计算梯度的形式 `gradients = tape.gradient(loss, weights)`

第二章,我们看到过 GradientTape 的输入可以是单一输入或者输入列表.可以输入常量或者更高维的张量.

到现在为止我们看到输入 tape.gradient() 都是由 tensorflow 生成的变量.但是实际上 tape.gradient() 可以接收任意的张量输入.但是默认情况下只有张量是可训练张量(trainable variables)时才会被跟踪??,对于冻结张量(常量)需要手动通过 `tape.watch()` 设置跟踪.(??有点云里雾里??)


In [ ]:
input_const = tf.constant(3.)

with tf.GradientTape() as tape:
    tape.watch(input_const)
    result = tf.square(input_const)

gradient = tape.gradient(result, input_const)

计算常量张量的梯度需要 `tape.watch` 设置跟踪.

计算梯度是非常消耗资源的事情,不可能预先跟踪所有张量.为了避免资源浪费,tensorflow 默认只监视可训练的张量(trainable variables),因为通常需要计算梯度的就是这些张量.

除了计算梯度以外,我们还可以计算梯度的梯度,物理学类比的化是计算速度,再计算加速度.


In [30]:
time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position = 4.9 * time**2 # ** 是乘方
    speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)
print(position)
print(speed)
print(acceleration)

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(9.8, shape=(), dtype=float32)


时间和位置: $p = 4.9 * time^2$

时间和速度: $s = 4.9 * 2 *time$

时间和加速度: $a = 4.9 *2$


## 端到端的 tensorflow 的分类器
